In [1]:
#import modules needed
import numpy as np 
import pandas as pd
import requests
from sqlalchemy import create_engine
import warnings

#make postgres connection
host = 'gispgdb.nrel.gov'
dbase = 'dav-gis'
user = 'kwaechte'
pwd = 'steeltree18'
con = create_engine('postgresql://{user}:{pwd}@{host}:5432/{dbase}'.format(host=host, dbase=dbase, user=user, pwd=pwd))

#define mapquest API configs (these are gds group configs)
api_key = 'RvaZ5PzcUBOwKphib8GGhgVVZFgZ06bxDKCXYhUc'
_app_id = 'gis-group'
API = 'https://internal-apis.nrel.gov/mapquest/geocoding/v1/address'

In [6]:
#### 4.1: pull in address data for data_source_1

sql = "select rid, name, address || ', ' || city || ' CA ' || zip as address from kwaechte.spotio_formatted where address is not null and city is not null and zip is not null and e0status is not null"
data_source_1_df = pd.read_sql(sql, con)
data_source_1_df.tail()

,rid,name,address
2057,35149,Maria Carpio,"984 Major Ave, Hayward CA 94542"
2058,76093,SUAD KARIC,"990 San Miguel Rd, Concord CA 94518"
2059,60845,Lori Hinckley,"992 Dowling Blvd, San Leandro CA 94577"
2060,19071,Dalibor John,"993 San Jose St, San Leandro CA 94577"
2061,8808,Test,"Sonata Drive, Fairfield CA 94534"


In [10]:
#loop over all records in dataframe to geocode
#
for i, row in data_source_1_df.iterrows():
    try: 
        address = row.address
        warnings.filterwarnings('ignore')
        #mapquest geocode api request
        r = requests.get(API, params={'api_key': api_key, '_app_id': _app_id, 'location': address}, verify=False)
        coords = r.json()['results'][0]['locations'][0]['displayLatLng']
        #write coordinates back to dataframe
        data_source_1_df.loc[i, 'latitude'] = coords['lat']
    #    data_source_1_df = data_source_1_df.append({'lat':i}, ignore_index=True)
        data_source_1_df.loc[i, 'longitude'] = coords['lng']
    #    data_source_1_df = data_source_1_df.append({'lng':i}, ignore_index=True)
    except: 
        print[i+" failed."]

#verify that the geocoding worked
data_source_1_df.head(5)

,rid,name,address,latitude,longitude
0,9451,Ngan K Du,"560 Branaugh Ct, Hayward CA 94544",37.624278,-122.057086
1,86806,Nancy Lopez,"1122 Sherman St, San Jose CA 95110",37.319030,-121.877980
2,32193,Binh T Nguyen,"27946 Pompano Ave, Hayward CA 94544",37.625999,-122.076816
3,9574,Noemi Velasquez,"1002 Thiel Rd, Hayward CA 94544",37.624005,-122.065666
4,3055,CAROL A SOTO,"10051 Bedford St, San Jose CA 95127",37.358145,-121.817481


In [14]:
print(len(data_source_1_df.loc[data_source_1_df.latitude.isnull()]))

data_source_1_df.tail(5)

,rid,name,address,latitude,longitude
2057,35149,Maria Carpio,"984 Major Ave, Hayward CA 94542",37.652510,-122.065428
2058,76093,SUAD KARIC,"990 San Miguel Rd, Concord CA 94518",37.955423,-122.022608
2059,60845,Lori Hinckley,"992 Dowling Blvd, San Leandro CA 94577",37.736651,-122.145540
2060,19071,Dalibor John,"993 San Jose St, San Leandro CA 94577",37.729790,-122.147158
2061,8808,Test,"Sonata Drive, Fairfield CA 94534",38.181166,-122.135254


In [19]:
### 8 write output table to postgres
import pandas as pd
out_table = 'geocoded_spotio_formatted_2061ct'
out_schema = 'kwaechte'
if_exists_decision = 'replace'
data_source_1_df.to_sql(out_table, con, schema=out_schema, if_exists=if_exists_decision)
print("export completed.")

export completed.
